In [2]:
!pip install TextBlob

     |████████████████████████████████| 636 kB 5.3 MB/s            


In [24]:
!pip install nltk

In [2]:
!pip install gensim

In [4]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests, zipfile, io
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
!pip install spacy

In [5]:
user_twt_pdf = pd.read_csv('gs://msca-bdp-students-bucket/shared_data/bhadri/user_twt_pdf.csv')
user_twt_pdf.head()

/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3012: DtypeWarning: Columns (0,1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,x_user_name,x_user_scrname,x_user_desc,x_user_location,x_user_id,covid_twt_count,total_tweet_count,avg_retwt_count,followers_count
0,0,"Robert W Malone, MD",RWMaloneMD,"Inventor of mRNA vaccines and RNA as a drug, B...","Madison, VA",3.984488e+07,27532.0,8521.0,1333.550305,473202.0
1,1,Disclose.tv,disclosetv,Observing world events unfold in the grand the...,NaN,1.539249e+07,21596.0,3789.0,2459.135812,736174.0
2,2,Karan Menon,TheKaranMenon,"comedian from jersey, woke tiktok king, featur...","Holmdel, NJ",1.049540e+18,16271.0,223.0,11724.467826,23226.0
3,3,Chuck Callesto,ChuckCallesto,Father and Former Candidate for Florida's 3rd ...,"Florida, USA",4.765364e+09,13755.0,31375.0,2092.251472,285801.0
4,4,윤서⁷,btsinthemoment,for #bts 아포방포 translations — 🇰🇷🇦🇺 he / prince ...,speedy,1.333743e+18,13675.0,4549.0,6298.398245,94746.0


In [7]:
user_twt_pdf[user_twt_pdf['covid_twt_count'] <= 5].shape

(356346, 10)

In [6]:
user_twt_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504577 entries, 0 to 504576
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         504576 non-null  object 
 1   x_user_name        396492 non-null  object 
 2   x_user_scrname     396506 non-null  object 
 3   x_user_desc        337762 non-null  object 
 4   x_user_location    258190 non-null  object 
 5   x_user_id          387990 non-null  float64
 6   covid_twt_count    387989 non-null  float64
 7   total_tweet_count  387979 non-null  float64
 8   avg_retwt_count    387979 non-null  float64
 9   followers_count    387979 non-null  float64
dtypes: float64(5), object(5)
memory usage: 38.5+ MB


In [41]:
user_twt_pdf[user_twt_pdf['followers_count'] < 25].sort_values('followers_count',ascending=False)

,Unnamed: 0,x_user_name,x_user_scrname,x_user_desc,x_user_location,x_user_id,covid_twt_count,total_tweet_count,avg_retwt_count,followers_count
322609,322596,Vandenack Weaver,VW_Law,Vandenack Weaver is a law firm in Nebraska and...,"Omaha, NE",1.276241e+18,1.0,236.0,0.0,24.0
204608,204598,EllaMenoPee2003,EllaPee2003,Just your everyday tweeter,NaN,1.295838e+18,1.0,14909.0,0.0,24.0
329026,329013,Raj Darji,RDarjiOfficial,Self taught entrepreneur trying to help people...,"New York, USA",8.505427e+17,1.0,194.0,0.0,24.0
283277,283265,Salty American,Michael10909626,Im a human .Part of the human race from planet...,Earth,1.075773e+18,1.0,7850.0,0.0,24.0
329015,329002,Lisa P,LisaPellegrin18,NaN,"Ontario, Canada",9.257666e+17,1.0,165.0,0.0,24.0
...,...,...,...,...,...,...,...,...,...,...
218736,218726,Restaurant Giant,RestaurantG10,Restaurant Giant - Restaurant News Site,NaN,1.449210e+18,1.0,92.0,0.0,0.0
218617,218607,Rolando Sanvicente,Rolafag01,NaN,NaN,1.467202e+18,1.0,1.0,0.0,0.0
218616,218606,Gordo,Gordo_FlyersRok,NaN,NaN,1.459896e+18,1.0,486.0,0.0,0.0
218602,218592,thomas crowley,thomasc99612034,NaN,NaN,1.467077e+18,1.0,9.0,0.0,0.0


In [6]:
user_twt_pdf = user_twt_pdf[user_twt_pdf['covid_twt_count'] > 5]
user_twt_pdf = user_twt_pdf[user_twt_pdf['followers_count'] > 10]
user_twt_pdf.shape

(30529, 10)

In [46]:
user_twt_pdf.sort_values('covid_twt_count',ascending=True)

,Unnamed: 0,x_user_name,x_user_scrname,x_user_desc,x_user_location,x_user_id,covid_twt_count,total_tweet_count,avg_retwt_count,followers_count
29567,29565,Rui Santos,neveshenriqu,NaN,NaN,1.465436e+18,6.0,11.0,0.000000,11.0
31341,31339,COMMONWEALTH NEWS CIC,CicCommonwealth,To encourage common sense news by combating hy...,commonwealthcentnews@gmail.com,1.017472e+18,6.0,412.0,0.000000,11.0
8700,8699,Orbital Affairs,Orbital_Affairs,NaN,Islamabad,1.306584e+18,21.0,3502.0,0.000000,11.0
4970,4969,Elon is God,500_cz,NaN,Hong Kong,1.419989e+18,39.0,7290.0,0.538462,11.0
27252,27250,Meenal gupta,Meenalgupta11,Dreamer♥️believer,India,3.284194e+09,6.0,845.0,2.500000,11.0
...,...,...,...,...,...,...,...,...,...,...
926,926,CNN Breaking News,cnnbrk,Breaking news from CNN Digital. Now 61M strong...,Everywhere,4.283330e+05,269.0,76826.0,163.033457,61712297.0
16803,16802,Selena Gomez,selenagomez,REVELACIÓN out now: http://smarturl.it/REVELAC...,Los Angeles,2.337569e+07,10.0,5333.0,3640.200000,65405780.0
6176,6175,Elon Musk,elonmusk,NaN,NaN,4.419640e+07,30.0,16218.0,21955.400000,65439112.0
3621,3620,Narendra Modi,narendramodi,Prime Minister of India,India,1.883978e+07,56.0,31301.0,5419.892857,73232559.0


In [7]:
user_desc = user_twt_pdf[['x_user_name','x_user_scrname','x_user_desc']]
user_desc.head()

,x_user_name,x_user_scrname,x_user_desc
0,"Robert W Malone, MD",RWMaloneMD,"Inventor of mRNA vaccines and RNA as a drug, B..."
1,Disclose.tv,disclosetv,Observing world events unfold in the grand the...
2,Karan Menon,TheKaranMenon,"comedian from jersey, woke tiktok king, featur..."
3,Chuck Callesto,ChuckCallesto,Father and Former Candidate for Florida's 3rd ...
4,윤서⁷,btsinthemoment,for #bts 아포방포 translations — 🇰🇷🇦🇺 he / prince ...


In [8]:
from gensim.parsing.preprocessing import remove_stopwords

def clean_text(text):
    '''
         clean the text from unwanted characters/text
    '''
    #cleaning the subtitles
    text = str(text)
    text = text.lower()
    text = re.sub('@[^\s]+','',text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # removes punctuations
    text = re.sub('\w*\d\w*', '', text) #removes numbers
    text = "".join(c for c in text if ord(c)<128)  # remove non ascii characters
    
    cleaned_text = remove_stopwords(text)
           
    return cleaned_text

In [9]:
for idx,row in user_desc.iterrows():
    user_desc.loc[idx, 'clean_desc'] = clean_text(row.loc['x_user_desc'])

/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [10]:
user_desc.head(25)

,x_user_name,x_user_scrname,x_user_desc,clean_desc
0,"Robert W Malone, MD",RWMaloneMD,"Inventor of mRNA vaccines and RNA as a drug, B...",inventor mrna vaccines rna drug bench bedside ...
1,Disclose.tv,disclosetv,Observing world events unfold in the grand the...,observing world events unfold grand theater time
2,Karan Menon,TheKaranMenon,"comedian from jersey, woke tiktok king, featur...",comedian jersey woke tiktok king featured cnn ...
3,Chuck Callesto,ChuckCallesto,Father and Former Candidate for Florida's 3rd ...,father candidate floridas congressional distri...
4,윤서⁷,btsinthemoment,for #bts 아포방포 translations — 🇰🇷🇦🇺 he / prince ...,bts translations prince backup
5,Dr. Eli David,DrEliDavid,"Researcher, Lecturer, and Founder | Co-Founder...",researcher lecturer founder cofounder deepcube...
6,Michael P Senger,MichaelPSenger,"Attorney, author of China’s Global Lockdown Pr...",attorney author chinas global lockdown propaga...
7,Sammy,TroubledSamm,artist,artist
8,CBS News,CBSNews,Your source for original reporting and trusted...,source original reporting trusted news
9,Breaking911,Breaking911,"Breaking News, Trending Stories & Everything T...",breaking news trending stories mainstream medi...


In [39]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.4 MB 5.9 kB/s               | 501.1 MB 72.1 MB/s eta 0:00:04
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [11]:
import spacy


In [12]:
def ner(entity):
    nlp = spacy.load("en_core_web_lg")
    text = nlp(entity)
    
    id_list = []
    for word in text.ents:
        id_list.append(word.label_)
        
    return id_list    

In [13]:
for idx,row in user_desc.iterrows():
    user_desc.loc[idx, 'identity'] = ner(row.loc['x_user_name'])

/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


ValueError: Must have equal len keys and value when setting with an iterable

In [14]:
user_desc[user_desc['x_user_name'].isnull()]

,x_user_name,x_user_scrname,x_user_desc,clean_desc,identity


## Cleanup

In [21]:
corpus = ""

In [22]:
for idx,row in user_desc.iterrows():
    corpus+=row.loc['clean_desc']

In [23]:
len(corpus)

2133451

In [27]:
wordlist = corpus.split()

KeyboardInterrupt: 

In [34]:
import collections

word_counter = collections.Counter(wordlist)
for word, count in word_counter.most_common(100):
    print(word, ": ", count)

news :  3503
health :  1479
account :  950
twitter :  869
love :  763
world :  751
politics :  721
follow :  693
people :  692
media :  642
life :  638
tweets :  588
dont :  570
author :  553
public :  548
sports :  537
new :  525
social :  507
political :  505
writer :  494
views :  493
free :  493
business :  492
fan :  464
science :  458
proud :  445
truth :  441
lover :  440
im :  425
research :  413
like :  406
trump :  405
breaking :  394
care :  383
covid :  381
editor :  380
music :  376
rights :  370
freedom :  368
latest :  364
local :  361
mom :  357
information :  357
human :  354
conservative :  353
time :  349
member :  348
community :  345
reporter :  343
best :  342
state :  333
family :  333
global :  332
good :  327
government :  326
god :  316
medical :  314
director :  312
national :  308
opinions :  304
president :  303
support :  303
daily :  302
email :  302
live :  302
university :  301
husband :  298
updates :  297
father :  297
dms :  292
country :  288
rts : 

In [29]:
def id_entity(desc):
    gov = ['government']
    health = ['health']
    news = ['news']
    individuals = ['influencer','author','journalist']
    

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [ ]:
for idx,row in user_desc.iterrows():
    user_desc.loc[idx, 'identity'] = ner(row.loc['x_user_name'])

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(ngram_range=(3,3))

# matrix of ngrams
ngrams = c_vec.fit_transform(user2['clean_desc'])

# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)

# list of ngrams
vocab = c_vec.vocabulary_
df_ngram3 = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'bigram/trigram'})

In [57]:
#to reduce corpus
user2 = user_desc.head(16000)

In [62]:
df_ngram.head(50)

,frequency,bigram/trigram
0,336,breaking news
1,307,official twitter
2,193,twitter account
3,144,public health
4,141,latest news
5,132,official account
6,105,news weather
7,88,social media
8,79,rts endorsements
9,76,husband father


In [64]:
df_ngram3.head(50)

,frequency,bigram/trigram
0,163,official twitter account
1,53,breaking news weather
2,52,official twitter handle
3,45,news weather sports
4,30,latest breaking news
5,26,official twitter page
6,24,official twitter feed
7,24,news weather traffic
8,22,source breaking news
9,20,pradesh congress committee
